In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

train_data = pd.read_csv('/kaggle/input/new-data-lun-csv/new_data_lun.csv')

In [3]:
same = train_data[train_data["is_same"]==1][:10000]
different = train_data[train_data["is_same"]==0][:12000]
new_train_data = pd.concat([same, different])

In [4]:
new_train_data["image_url3"] = list(new_train_data["image_url1"].values)[::-1]
new_train_data = new_train_data[new_train_data["is_same"]==1]
new_train_data.head()

,image_url1,image_url2,is_same,image_url3
1,965225293.jpg,965564035.jpg,1,909399908.jpg
3,917878082.jpg,921610429.jpg,1,926015250.jpg
4,941374542.jpg,941588763.jpg,1,938000122.jpg
8,925692435.jpg,925916250.jpg,1,930156741.jpg
13,945545057.jpg,945547680.jpg,1,892382884.jpg


In [5]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageOps
from six import BytesIO
import tensorflow as tf
import tensorflow_hub as hub

In [6]:
train_data = new_train_data

In [7]:
dir_img_1 = '/kaggle/input/traindataset/LUN_DataSet/train_url1'
dir_img_2 = '/kaggle/input/traindataset/LUN_DataSet/train_url2'

In [8]:
from tensorflow.keras import Input, Sequential, Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import applications, layers, losses, optimizers, metrics

mobile_net = VGG16(
    weights="imagenet", input_shape=(224,224, 3), include_top=False
)

flatten = layers.Flatten()(mobile_net.output)
x1 = layers.Dense(512, activation="relu")(flatten)
x1 = layers.BatchNormalization()(x1)
x2 = layers.Dense(256, activation="relu")(x1)
x2 = layers.BatchNormalization()(x2)
output = layers.Dense(256)(x2)

embedding = Model(mobile_net.input, output, name="Embedding")

trainable = False
for layer in mobile_net.layers:
    if layer.name == "block5_conv1":
        trainable = True
    layer.trainable = trainable

58889256/58889256 [==============================] - 0s 0us/step


In [9]:
embedding.summary()

Model: "Embedding"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0 

In [10]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Lambda
import tensorflow.keras.backend as K


class TripletModel(Model):
    def __init__(self):
        super(TripletModel, self).__init__()
        self.embedding = embedding

    def call(self, inputs):
        anchor, positive, negative = inputs
        anchor_embedding = self.embedding(anchor)
        positive_embedding = self.embedding(positive)
        negative_embedding = self.embedding(negative)
        return anchor_embedding, positive_embedding, negative_embedding

In [11]:
def cosine_similarity_batched(x1, x2):
    # Compute L2-norm of each vector
    x1_norm = tf.linalg.norm(x1, axis=1, keepdims=True)
    x2_norm = tf.linalg.norm(x2, axis=1, keepdims=True)
    # Compute dot product between vectors
    dot_product = tf.reduce_sum(x1 * x2, axis=1, keepdims=True)
    # Compute cosine similarity
    cosine_similarity = dot_product / (x1_norm * x2_norm)

    return cosine_similarity

In [12]:
def custom_f1_score_pos(y_true, y_pred):
    # Calculate true positives, false positives, and false negatives
    TP = tf.math.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 1), tf.equal(y_pred, 1)), dtype=tf.float32), axis=0)
    FP = tf.math.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(y_pred, 1)), dtype=tf.float32), axis=0)
    FN = tf.math.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 1), tf.equal(y_pred, 0)), dtype=tf.float32), axis=0)

    # Calculate precision, recall, and F1 score
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

In [13]:
def custom_f1_score_neg(y_true, y_pred):
    # Calculate true positives, false positives, and false negatives
    TP = tf.math.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(y_pred, 0)), dtype=tf.float32), axis=0)
    FP = tf.math.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(y_pred, 1)), dtype=tf.float32), axis=0)
    FN = tf.math.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 1), tf.equal(y_pred, 0)), dtype=tf.float32), axis=0)

    # Calculate precision, recall, and F1 score
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import pandas as pd
import numpy as np
import os


class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, dataframe, batch_size=32, image_size=(224, 224)):
        self.dataframe = dataframe
        self.batch_size = batch_size
        self.image_size = image_size
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.dataframe) / self.batch_size))

    def __getitem__(self, index):
        batch_df = self.dataframe[index * self.batch_size:(index + 1) * self.batch_size]

        batch_images1 = []
        batch_images2 = []
        batch_images3 = []
        for _, row in batch_df.iterrows():
            img1_url, img2_url, img3_url = row['image_url1'], row['image_url2'], row['image_url3']
            image1 = Image.open(os.path.join(dir_img_1, img1_url))
            image2 = Image.open(os.path.join(dir_img_2, img2_url))
            image3 = 0
            try:
                image3 = Image.open(os.path.join(dir_img_1, img3_url))
            except FileNotFoundError:
                image3 = Image.open(os.path.join(dir_img_2, img3_url))
                
                
            if image1.mode == "L" or image1.mode == "RGBA":
                image1 = image1.convert("RGB")
            if image2.mode == "L" or image2.mode == "RGBA":
                image2 = image2.convert("RGB")
            if image3.mode == "L" or image3.mode == "RGBA":
                image3 = image3.convert("RGB")
            
            image1 = image1.resize((224, 224))  
            image2 = image2.resize((224, 224)) 
            image3 = image3.resize((224, 224))
                
            image1 = tf.convert_to_tensor(image1)
            image2 = tf.convert_to_tensor(image2)
            image3 = tf.convert_to_tensor(image3)

            batch_images1.append(image1)
            batch_images2.append(image2)
            batch_images3.append(image3)

        batch_images1 = np.array(batch_images1) / 255.0
        batch_images2 = np.array(batch_images2) / 255.0
        batch_images3 = np.array(batch_images3) /  255.0
        return batch_images1, batch_images2, batch_images3

    def on_epoch_end(self):
        self.dataframe = self.dataframe.sample(frac=1)

In [15]:
threshold = 0.77

In [16]:
def custom_loss(labels, predictions):
    # Compute cosine distances between anchor, positive, and negative examples
    anchor_embedding, positive_embedding, negative_embedding = predictions
    pos_sim = cosine_similarity_batched(anchor_embedding, positive_embedding)
    neg_sim = cosine_similarity_batched(anchor_embedding, negative_embedding)
    pos_dist = 1 - pos_sim
    neg_dist = 1 - neg_sim
    pos_f1 =  custom_f1_score_pos(tf.expand_dims(labels[1], axis=1), tf.cast(pos_sim  > threshold, tf.int32))
    neg_f1 = custom_f1_score_neg(tf.expand_dims(labels[0], axis=1), tf.cast(neg_sim  > threshold, tf.int32))
    alpha = 1
    return K.mean(K.maximum(pos_dist - neg_dist + alpha, 0.0)), (pos_f1 + neg_f1)/2

In [17]:
weights = []
best_metric = np.inf

In [18]:
def train_model(model, train_dataset, val_dataset):
    # Define loss function, optimizer, and metrics
    loss_fn = custom_loss
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    train_loss_metric = 0.0
    val_loss_metric = 0.0
    train_f1_metric =  tf.keras.metrics.Mean()
    val_f1_metric =  tf.keras.metrics.Mean()
    epochs = 10

    @tf.function
    def train_step(x1, x2, x3):
        with tf.GradientTape() as tape:
            # Forward pass
            predictions = model([x1, x2, x3])
            labels = [j for j in [0,1] for i in range(x1.shape[0])]
            labels = tf.constant(labels)
            labels = tf.reshape(labels, (2,x1.shape[0]))
            loss_value, train_f1 = loss_fn(labels, predictions)

        # Backward pass
        gradients = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        return loss_value, train_f1

    @tf.function
    def val_step(x1, x2, x3):
        # Forward pass
        predictions = model([x1, x2, x3])
        labels = [j for j in [0,1] for i in range(x1.shape[0])]
        labels = tf.constant(labels)
        labels = tf.reshape(labels, (2,x1.shape[0]))
        loss_value, val_f1 = loss_fn(labels, predictions)
        return loss_value, val_f1

    # Training loop
    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        # Reset metrics
        train_loss_metric = 0.0
        val_loss_metric = 0.0
        train_f1_metric.reset_states()
        val_f1_metric.reset_states()
        train_k = 0
        val_k = 0

        # Training
        for x1, x2, x3 in train_dataset:
            loss_value, train_f1 = train_step(x1, x2, x3)
            train_loss_metric += loss_value
            train_k += 1
            train_f1_metric.update_state(train_f1)
        
        # Validation
        for x1, x2, x3 in val_dataset:
            loss_value, val_f1 = val_step(x1, x2, x3)
            val_loss_metric += loss_value
            val_k += 1
            val_f1_metric.update_state(val_f1)
            
        train_loss_metric /= (train_k)
        val_loss_metric /= (val_k)
        
        # Record the epoch losses
        train_losses.append(train_loss_metric)
        val_losses.append(val_loss_metric)
        
        global best_metric
        
        if val_loss_metric < best_metric:
            best_metric = val_loss_metric
            model.save('/kaggle/working/best_model', save_format='tf')
            global weights
            weights = model.layers[0].get_weights()  

        # Print progress
        print(f"Epoch {epoch+1}/{epochs}:")
        print(f"Train Loss: {train_loss_metric}, Train F1: {train_f1_metric.result().numpy()}")
        print(f"Val Loss: {val_loss_metric}, Val F1: {val_f1_metric.result().numpy()}")
        print()

    return train_losses, val_losses

In [19]:
# # detect and init the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

#  # instantiate a distribution strategy
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [20]:
model = TripletModel()

In [ ]:
# Train the model
from sklearn.model_selection import KFold

# Define the number of folds for cross-validation
num_folds = 2

# Initialize the K-Fold object
kf = KFold(n_splits=num_folds, shuffle=True)

# Initialize lists to store the training histories for each fold
train_histories = []
val_histories = []

# Perform cross-validation
for train_index, val_index in kf.split(train_data):
    train_dataframe = train_data.iloc[train_index]
    val_dataframe = train_data.iloc[val_index]

    train_data_generator = DataGenerator(train_dataframe, batch_size=32)
    val_data_generator = DataGenerator(val_dataframe, batch_size=32)

    train_dataset = tf.data.Dataset.from_generator(
        lambda: train_data_generator,
        output_signature=(
            tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
             tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32)
        )
    )

    val_dataset = tf.data.Dataset.from_generator(
        lambda: val_data_generator,
        output_signature=(
            tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
             tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
           tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32)
        )
    )
    # Perform the custom model training
    train_losses, val_losses = train_model(model, train_dataset, val_dataset)

    # Store the training histories for each fold
    train_histories.append(train_losses)
    val_histories.append(val_losses)

Epoch 1/10:
Train Loss: 0.21746133267879486, Train F1: nan
Val Loss: 0.18608200550079346, Val F1: 0.9958697557449341

Epoch 2/10:
Train Loss: 0.059792980551719666, Train F1: 0.9954248666763306
Val Loss: 0.08521177619695663, Val F1: 0.9913681149482727

Epoch 3/10:
Train Loss: 0.020478516817092896, Train F1: 0.9976146817207336
Val Loss: 0.07740872353315353, Val F1: 0.9913020133972168

Epoch 4/10:
Train Loss: 0.006604009307920933, Train F1: 0.9986353516578674
Val Loss: 0.0808311179280281, Val F1: 0.9917841553688049

Epoch 5/10:
Train Loss: 0.002744462573900819, Train F1: 0.9988847374916077
Val Loss: 0.07847895473241806, Val F1: 0.99183189868927

Epoch 6/10:
Train Loss: 0.0012684103567153215, Train F1: 0.9993935227394104
Val Loss: 0.07602201402187347, Val F1: 0.9920370578765869



In [ ]:
print(weights)

In [ ]:
import matplotlib.pyplot as plt

# Plot the training results for each fold
for fold, (train_losses, val_losses) in enumerate(zip(train_histories, val_histories)):
    # Plot loss curves
    plt.plot(train_losses, label=f'Train ')
    plt.plot(val_losses, label=f'Validation ')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()
plt.show()